In [ ]:
import numpy as np
import pandas as pd
import xgboost as xgb
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn import tree
import datetime
from google.colab import files

In [ ]:
LOANS_TRAIN = pd.read_csv('https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/miniprojects/LOANS_TRAIN.csv')
LOANS_TEST = pd.read_csv('https://raw.githubusercontent.com/charlesincharge/Caltech-CS155-2022/main/miniprojects/LOANS_TEST.csv')

In [ ]:
LOANS_TRAIN

,id,loan_amnt,term_(months),int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,purpose,title,zip_code,addr_state,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,loan_status
0,0,5000,36,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,credit_card,Computer,860xx,AZ,27.65,Jan-1985,3,0,13648,83.7%,9,f,Individual,NaN,0.0,Fully Paid
1,1,2500,60,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,car,bike,309xx,GA,1.00,Apr-1999,3,0,1687,9.4%,4,f,Individual,NaN,0.0,Charged Off
2,2,2400,36,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,small_business,real estate business,606xx,IL,8.72,Nov-2001,2,0,2956,98.5%,10,f,Individual,NaN,0.0,Fully Paid
3,3,10000,36,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,other,personel,917xx,CA,20.00,Feb-1996,10,0,5598,21%,37,f,Individual,NaN,0.0,Fully Paid
4,4,3000,60,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,other,Personal,972xx,OR,17.94,Jan-1996,15,0,27783,53.9%,38,f,Individual,NaN,0.0,Fully Paid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197245,199995,1500,36,7.90%,46.94,A,A4,USAF,5 years,RENT,42000.0,Source Verified,Sep-2012,other,Auto Repairs,890xx,NV,18.23,Aug-2005,5,0,3597,29.5%,10,f,Individual,0.0,0.0,Fully Paid
197246,199996,14075,36,8.90%,446.93,A,A5,NaN,NaN,RENT,36000.0,Verified,Sep-2012,credit_card,Credit card refinancing,891xx,NV,22.20,Jun-1972,15,0,17473,47.1%,24,f,Individual,0.0,0.0,Fully Paid
197247,199997,11000,36,14.09%,376.44,B,B5,Gateway Regional Medical Center,2 years,RENT,45000.0,Not Verified,Sep-2012,debt_consolidation,Debt Consolidation,631xx,MO,8.91,Oct-2006,18,0,8956,43.7%,20,f,Individual,0.0,0.0,Fully Paid
197248,199998,31400,60,22.95%,884.28,F,F1,Navy Federal Credit Union,3 years,MORTGAGE,85000.0,Verified,Sep-2012,debt_consolidation,debt free plan!,220xx,VA,24.76,Oct-1999,12,0,23150,77.4%,23,f,Individual,4.0,0.0,Charged Off


In [ ]:
train_data = pd.DataFrame()
#Keep loan amount the same
train_data['loan_amnt'] = LOANS_TRAIN['loan_amnt']
#Keep loan term the same, only has 36 or 60 so could do 0/1: [0 if x == 36 else 1 for x in LOANS_TRAIN['term_(months)']]
train_data['term'] = LOANS_TRAIN['term_(months)']
#Convert interest rate to a float
train_data['int_rate'] = [float(x.strip(" ").strip("%")) for x in LOANS_TRAIN['int_rate']]
#Keep installment the same
train_data['installment'] = LOANS_TRAIN['installment']
#Change grade from A-G to 0-6
train_data['grade'] = [ord(x)-65 for x in LOANS_TRAIN['grade']]
#Change subgrade from A1-G5 to 0-4,10-14,..,60-64
train_data['sub_grade'] = [10*(ord(x[0])-65)+int(x[1])-1 for x in LOANS_TRAIN['sub_grade']]
#Dropping emp_title column because 119834 unique out of 197249 total
#Convert emp length from <1, 1, ..., 9, 10+ to 0-10, nan value coded as -1 should decide what to do with nan
train_data['emp_length'] = [-1 if pd.isna(x) else 0 if '<' in x else int(x.strip(" years").strip("+")) for x in LOANS_TRAIN['emp_length']]
#Home ownership has rent, own, mortgage, other, none, so making a column for each of these
#better than a single column with these coded as numbers??
train_data['ownership_rent'] = [1 if x == 'RENT' else 0 for x in LOANS_TRAIN['home_ownership']]
train_data['ownership_own'] = [1 if x == 'OWN' else 0 for x in LOANS_TRAIN['home_ownership']]
train_data['ownership_mortgage'] = [1 if x == 'MORTGAGE' else 0 for x in LOANS_TRAIN['home_ownership']]
train_data['ownership_other'] = [1 if x == 'OTHER' else 0 for x in LOANS_TRAIN['home_ownership']]
train_data['ownership_none'] = [1 if x == 'NONE' else 0 for x in LOANS_TRAIN['home_ownership']]
#Keep annual income the same
train_data['annual_inc'] = LOANS_TRAIN['annual_inc']
#Verification status has verified, source verified, not verified, so making a column for each of these
#better than a single column with these coded as numbers?? could also do single column 0 for not verified 1 for other two
train_data['status_verified'] = [1 if x == 'Verified' else 0 for x in LOANS_TRAIN['verification_status']]
train_data['status_source_verified'] = [1 if x == 'Source Verified' else 0 for x in LOANS_TRAIN['verification_status']]
train_data['status_not_verified'] = [1 if x == 'Not Verified' else 0 for x in LOANS_TRAIN['verification_status']]
#Not sure what to do with issue date so split into issue year and issue month
#Issue year changed from 2007-2013 to 0-6
train_data['issue_year'] = [int(x[-4:])-2007 for x in LOANS_TRAIN['issue_d']]
#Issue month changed from jan-dec to 0-11
train_data['issue_month'] = [datetime.datetime.strptime(x[:3], "%b").month-1 for x in LOANS_TRAIN['issue_d']]
#Purpose has multiple options so making a column for each of these
train_data['purpose_credit_card'] = [1 if x == 'credit_card' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_car'] = [1 if x == 'car' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_small_business'] = [1 if x == 'small_business' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_other'] = [1 if x == 'other' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_wedding'] = [1 if x == 'wedding' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_debt_consolidation'] = [1 if x == 'debt_consolidation' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_home_improvement'] = [1 if x == 'home_improvement' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_major_purchase'] = [1 if x == 'major_purchase' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_medical'] = [1 if x == 'medical' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_moving'] = [1 if x == 'moving' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_vacation'] = [1 if x == 'vacation' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_house'] = [1 if x == 'house' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_renewable_energy'] = [1 if x == 'renewable_energy' else 0 for x in LOANS_TRAIN['purpose']]
train_data['purpose_educational'] = [1 if x == 'educational' else 0 for x in LOANS_TRAIN['purpose']]
#Dropping title column because 53461 unique out of 197249 total
#Dropping zip_code column because 855 unique out of 197249 total, could also do numerical as below
# train_data['zip_code'] = [int(x[:3]) for x in LOANS_TRAIN['zip_code']]
#Dropping state column because idk what to do for this
#Keep dti the same
train_data['dti'] = LOANS_TRAIN['dti']
#Not sure what to do with issue date so split into issue year and issue month
#Issue year changed from 1946-2010 to 0-64
train_data['cr_line_year'] = [int(x[-4:])-1946 for x in LOANS_TRAIN['earliest_cr_line']]
#Issue month changed from jan-dec to 0-11
train_data['cr_line_month'] = [datetime.datetime.strptime(x[:3], "%b").month-1 for x in LOANS_TRAIN['earliest_cr_line']]
#Keep open_acc the same
train_data['open_acc'] = LOANS_TRAIN['open_acc']
#Keep pub_rec the same
train_data['pub_rec'] = LOANS_TRAIN['pub_rec']
#Keep revol_bal the same
train_data['revol_bal'] = LOANS_TRAIN['revol_bal']
#Convert revol_util to a float
train_data['revol_util'] = [-1 if pd.isna(x) else float(x.strip("%")) for x in LOANS_TRAIN['revol_util']]
#Keep total_acc the same
train_data['total_acc'] = LOANS_TRAIN['total_acc']
#Convert initial_list_status from f/w to 0/1
train_data['initial_list_status'] = [0 if x == 'f' else 1 for x in LOANS_TRAIN['initial_list_status']]
#Convert nan values to -1 for pub_rec_bankruptcies
train_data['pub_bankruptcies'] = [-1 if pd.isna(x) else x for x in LOANS_TRAIN['pub_rec_bankruptcies']]
#Dropping application type because all of them are individual
#Convert nan values to -1 for mort_acc
train_data['mort_acc'] = [-1 if pd.isna(x) else x for x in LOANS_TRAIN['mort_acc']]
#For now making our loan_status with 1 as charged off and 0 as fully paid
train_data['loan_status'] = [1 if x == 'Charged Off' else 0 for x in LOANS_TRAIN['loan_status']]
#Setting intermediate states as the original states
train_data['addr_state'] = LOANS_TRAIN['addr_state']
#Setting each state value to be equal to the average of all thats states loan status (0/1)
state_means = {}
for x in train_data['addr_state'].unique():
    state_means[x] = train_data[train_data['addr_state']==x]['loan_status'].mean()
train_data['addr_state'] = [state_means[x] for x in train_data['addr_state']]
train_data.drop('loan_status', inplace=True, axis=1)

train_labels = pd.DataFrame()
#For now making our loan_status with 0 as charged off and 1 as fully paid
train_labels['loan_status'] = [1 if x == 'Charged Off' else 0 for x in LOANS_TRAIN['loan_status']]

In [ ]:
test_data = pd.DataFrame()
#Keep loan amount the same
test_data['loan_amnt'] = LOANS_TEST['loan_amnt']
#Keep loan term the same, only has 36 or 60 so could do 0/1: [0 if x == 36 else 1 for x in LOANS_TRAIN['term_(months)']]
test_data['term'] = LOANS_TEST['term_(months)']
#Convert interest rate to a float
test_data['int_rate'] = [float(x.strip(" ").strip("%")) for x in LOANS_TEST['int_rate']]
#Keep installment the same
test_data['installment'] = LOANS_TEST['installment']
#Change grade from A-G to 0-6
test_data['grade'] = [ord(x)-65 for x in LOANS_TEST['grade']]
#Change subgrade from A1-G5 to 0-4,10-14,..,60-64
test_data['sub_grade'] = [10*(ord(x[0])-65)+int(x[1])-1 for x in LOANS_TEST['sub_grade']]
#Dropping emp_title column because 119834 unique out of 197249 total
#Convert emp length from <1, 1, ..., 9, 10+ to 0-10, nan value coded as -1 should decide what to do with nan
test_data['emp_length'] = [-1 if pd.isna(x) else 0 if '<' in x else int(x.strip(" years").strip("+")) for x in LOANS_TEST['emp_length']]
#Home ownership has rent, own, mortgage, other, none, so making a column for each of these
#better than a single column with these coded as numbers??
test_data['ownership_rent'] = [1 if x == 'RENT' else 0 for x in LOANS_TEST['home_ownership']]
test_data['ownership_own'] = [1 if x == 'OWN' else 0 for x in LOANS_TEST['home_ownership']]
test_data['ownership_mortgage'] = [1 if x == 'MORTGAGE' else 0 for x in LOANS_TEST['home_ownership']]
test_data['ownership_other'] = [1 if x == 'OTHER' else 0 for x in LOANS_TEST['home_ownership']]
test_data['ownership_none'] = [1 if x == 'NONE' else 0 for x in LOANS_TEST['home_ownership']]
#Keep annual income the same
test_data['annual_inc'] = LOANS_TEST['annual_inc']
#Verification status has verified, source verified, not verified, so making a column for each of these
#better than a single column with these coded as numbers?? could also do single column 0 for not verified 1 for other two
test_data['status_verified'] = [1 if x == 'Verified' else 0 for x in LOANS_TEST['verification_status']]
test_data['status_source_verified'] = [1 if x == 'Source Verified' else 0 for x in LOANS_TEST['verification_status']]
test_data['status_not_verified'] = [1 if x == 'Not Verified' else 0 for x in LOANS_TEST['verification_status']]
#Not sure what to do with issue date so split into issue year and issue month
#Issue year changed from 2007-2013 to 0-6
test_data['issue_year'] = [int(x[-4:])-2007 for x in LOANS_TEST['issue_d']]
#Issue month changed from jan-dec to 0-11
test_data['issue_month'] = [datetime.datetime.strptime(x[:3], "%b").month-1 for x in LOANS_TEST['issue_d']]
#Purpose has multiple options so making a column for each of these
test_data['purpose_credit_card'] = [1 if x == 'credit_card' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_car'] = [1 if x == 'car' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_small_business'] = [1 if x == 'small_business' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_other'] = [1 if x == 'other' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_wedding'] = [1 if x == 'wedding' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_debt_consolidation'] = [1 if x == 'debt_consolidation' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_home_improvement'] = [1 if x == 'home_improvement' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_major_purchase'] = [1 if x == 'major_purchase' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_medical'] = [1 if x == 'medical' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_moving'] = [1 if x == 'moving' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_vacation'] = [1 if x == 'vacation' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_house'] = [1 if x == 'house' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_renewable_energy'] = [1 if x == 'renewable_energy' else 0 for x in LOANS_TEST['purpose']]
test_data['purpose_educational'] = [1 if x == 'educational' else 0 for x in LOANS_TEST['purpose']]
#Dropping title column because 53461 unique out of 197249 total
#Dropping zip_code column because 855 unique out of 197249 total, could also do numerical as below
# train_data['zip_code'] = [int(x[:3]) for x in LOANS_TRAIN['zip_code']]
#Dropping state column because idk what to do for this
#Keep dti the same
test_data['dti'] = [-1 if pd.isna(x) else x for x in LOANS_TEST['dti']]
#Not sure what to do with issue date so split into issue year and issue month
#Issue year changed from 1946-2010 to 0-64
test_data['cr_line_year'] = [int(x[-4:])-1946 for x in LOANS_TEST['earliest_cr_line']]
#Issue month changed from jan-dec to 0-11
test_data['cr_line_month'] = [datetime.datetime.strptime(x[:3], "%b").month-1 for x in LOANS_TEST['earliest_cr_line']]
#Keep open_acc the same
test_data['open_acc'] = LOANS_TEST['open_acc']
#Keep pub_rec the same
test_data['pub_rec'] = LOANS_TEST['pub_rec']
#Keep revol_bal the same
test_data['revol_bal'] = LOANS_TEST['revol_bal']
#Convert revol_util to a float
test_data['revol_util'] = [-1 if pd.isna(x) else float(x.strip("%")) for x in LOANS_TEST['revol_util']]
#Keep total_acc the same
test_data['total_acc'] = LOANS_TEST['total_acc']
#Convert initial_list_status from f/w to 0/1
test_data['initial_list_status'] = [0 if x == 'f' else 1 for x in LOANS_TEST['initial_list_status']]
#Convert nan values to -1 for pub_rec_bankruptcies
test_data['pub_bankruptcies'] = [-1 if pd.isna(x) else x for x in LOANS_TEST['pub_rec_bankruptcies']]
#Dropping application type because all of them are individual
#Convert nan values to -1 for mort_acc
test_data['mort_acc'] = [-1 if pd.isna(x) else x for x in LOANS_TEST['mort_acc']]
#Setting intermediate states as the original states
test_data['addr_state'] = LOANS_TEST['addr_state']
#Setting each state value to be equal to the average of all thats states loan status (0/1)
test_data['addr_state'] = [state_means[x] if x in state_means else train_labels['loan_status'].mean() for x in test_data['addr_state']]

In [ ]:
train_data

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_length,ownership_rent,ownership_own,ownership_mortgage,ownership_other,ownership_none,annual_inc,status_verified,status_source_verified,status_not_verified,issue_year,issue_month,purpose_credit_card,purpose_car,purpose_small_business,purpose_other,purpose_wedding,purpose_debt_consolidation,purpose_home_improvement,purpose_major_purchase,purpose_medical,purpose_moving,purpose_vacation,purpose_house,purpose_renewable_energy,purpose_educational,dti,cr_line_year,cr_line_month,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,pub_bankruptcies,mort_acc,addr_state
0,5000,36,10.65,162.87,1,11,10,1,0,0,0,0,24000.0,1,0,0,4,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,27.65,39,0,3,0,13648,83.7,9,0,0.0,-1.0,0.151597
1,2500,60,15.27,59.83,2,23,0,1,0,0,0,0,30000.0,0,1,0,4,11,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1.00,53,3,3,0,1687,9.4,4,0,0.0,-1.0,0.141727
2,2400,36,15.96,84.33,2,24,10,1,0,0,0,0,12252.0,0,0,1,4,11,0,0,1,0,0,0,0,0,0,0,0,0,0,0,8.72,55,10,2,0,2956,98.5,10,0,0.0,-1.0,0.140521
3,10000,36,13.49,339.31,2,20,10,1,0,0,0,0,49200.0,0,1,0,4,11,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20.00,50,1,10,0,5598,21.0,37,0,0.0,-1.0,0.149500
4,3000,60,12.69,67.79,1,14,1,1,0,0,0,0,80000.0,0,1,0,4,11,0,0,0,1,0,0,0,0,0,0,0,0,0,0,17.94,50,0,15,0,27783,53.9,38,0,0.0,-1.0,0.137840
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
197245,1500,36,7.90,46.94,0,3,5,1,0,0,0,0,42000.0,0,1,0,5,8,0,0,0,1,0,0,0,0,0,0,0,0,0,0,18.23,59,7,5,0,3597,29.5,10,0,0.0,0.0,0.190010
197246,14075,36,8.90,446.93,0,4,-1,1,0,0,0,0,36000.0,1,0,0,5,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,22.20,26,5,15,0,17473,47.1,24,0,0.0,0.0,0.190010
197247,11000,36,14.09,376.44,1,14,2,1,0,0,0,0,45000.0,0,0,1,5,8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,8.91,60,9,18,0,8956,43.7,20,0,0.0,0.0,0.161613
197248,31400,60,22.95,884.28,5,50,3,0,0,1,0,0,85000.0,1,0,0,5,8,0,0,0,0,0,1,0,0,0,0,0,0,0,0,24.76,53,9,12,0,23150,77.4,23,0,0.0,4.0,0.155638


In [ ]:
train_labels = np.array(train_labels['loan_status'])
train_features = np.array(train_data)
test_features = np.array(test_data)

In [ ]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)

Training Features Shape: (197250, 44)
Training Labels Shape: (197250,)
Testing Features Shape: (42480, 44)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
n_estimators = [600]
max_features = ['sqrt']
max_depth = [None]
# max_depth.append(None)
# min_samples_split = [2, 5, 10]
random_state = [50]
min_samples_leaf = [30]
# bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
            #    'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'random_state': random_state
            #    'bootstrap': bootstrap
}
print(random_grid)

{'n_estimators': [600], 'max_features': ['sqrt'], 'max_depth': [None], 'min_samples_leaf': [30], 'random_state': [None, 50]}


In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(max_features='sqrt',oob_score=True)  

In [ ]:
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, n_jobs = -1,
                               random_state=21,return_train_score=True,scoring='roc_auc')
rf_random.fit(train_features, train_labels)

RandomizedSearchCV(cv=3,
                   estimator=RandomForestRegressor(max_features='sqrt',
                                                   oob_score=True),
                   n_iter=2, n_jobs=-1,
                   param_distributions={'max_depth': [None],
                                        'max_features': ['sqrt'],
                                        'min_samples_leaf': [30],
                                        'n_estimators': [600],
                                        'random_state': [None, 50]},
                   random_state=21, return_train_score=True, scoring='roc_auc')

In [ ]:
rf_random.best_params_

{'max_depth': None,
 'max_features': 'sqrt',
 'min_samples_leaf': 30,
 'n_estimators': 600,
 'random_state': 50}

In [ ]:
rf_random.cv_results_

{'mean_fit_time': array([205.45531209, 201.92741219]),
 'mean_score_time': array([8.10164841, 8.21128758]),
 'mean_test_score': array([0.68947154, 0.6895478 ]),
 'mean_train_score': array([0.82313563, 0.82323769]),
 'param_max_depth': masked_array(data=[None, None],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_max_features': masked_array(data=['sqrt', 'sqrt'],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_min_samples_leaf': masked_array(data=[30, 30],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_n_estimators': masked_array(data=[600, 600],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'param_random_state': masked_array(data=[None, 50],
              mask=[False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': None,
   'max_features': 'sqrt',
   'min_samples_lea

In [ ]:
rpredictions = rf_random.predict(test_features)
np.mean(rpredictions)

0.14674424903671893

In [ ]:
rf = RandomForestRegressor(n_estimators = 600,oob_score=True,random_state=50,max_features='sqrt',min_samples_leaf=30,n_jobs=-1)
rf.fit(train_features, train_labels);

In [ ]:
predictions = rf.predict(test_features)
np.mean(predictions)

0.14689650260458928

In [ ]:
out = pd.DataFrame({'id':np.array([i for i in range(200000,200000+len(predictions))]),
       'loan_status': predictions})
out

,id,loan_status
0,200000,0.012056
1,200001,0.181228
2,200002,0.051488
3,200003,0.114940
4,200004,0.108310
...,...,...
42475,242475,0.278911
42476,242476,0.203773
42477,242477,0.329231
42478,242478,0.133267


In [ ]:
out.to_csv("just_for_fun_randforest_600_trees.csv",index=False)